# Page 18 (Lokeshwar): Bermudan put on a single asset

In [ ]:
from DiscretizationSchemes import discretizationScheme, StockpriceGBM # consider different discretization schemes?
from intrinsicOptionValues import call, put # consider more payoffs?

from helpers import forward, bs_put, bs_call
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [ ]:
# set parameters
S0, K, r, sigma, T, M = 40, 40, 0.06, 0.2, 1, 10
p = 32
mu = r # changing this should not make a difference??

In [ ]:
# compute continuation value Qtm (for ease of notation we write tm where the book writes t_{m-1})
def Qt_mminus1(beta_tm, S_tm, Deltat_mplus1):
  """
  Deltat_mplus1 = t_{m+1} - t_m
  """
  sumCondExp = beta_tm[3*p]
  for i in range(p):
    w_i, b_i, omega_i = beta_tm[i], beta_tm[p+i], beta_tm[2*p+i]
    if w_i >= 0 and b_i >= 0:
      condExp = forward(w_i*S_tm + b_i, r, Deltat_mplus1)
    elif w_i >= 0 and b_i < 0:
      condExp = w_i * bs_call(S_tm, -b_i/w_i, Deltat_mplus1, r, sigma)
    elif w_i < 0 and b_i >= 0:
      condExp = -w_i * bs_put(S_tm, -b_i/w_i, Deltat_mplus1, r, sigma)
    elif w_i < 0 and b_i < 0:
      condExp = 0
    sumCondExp += omega_i*condExp
  return sumCondExp


In [ ]:
monitoring_dates = np.linspace(0,T,M+1)[1:]
N = 10000

In [ ]:
def fitBeta(S_tm, V_tm, load_weights):

  # initialize model
  model = keras.Sequential()

  # input layer
  kernel_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  bias_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  model.add(keras.layers.Dense(p, input_dim=1, kernel_initializer=kernel_init, use_bias=True, bias_initializer=bias_init, activation='relu'))

  # output layer
  kernel_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  bias_init = tf.keras.initializers.RandomUniform(minval=0., maxval=1.)
  model.add(keras.layers.Dense(1, kernel_initializer=kernel_init, use_bias=True, bias_initializer=bias_init, activation='linear'))
  
  # optimize using Adam
  opt = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=opt,loss='mean_squared_error')

  # reuse weights from previous timestep
  if load_weights==True:
    model.load_weights('model_weights.h5', by_name=False)

  # fit the model
  x = S_tm
  y = V_tm
  model.fit(x, y, epochs=1000, batch_size=int(N/10))

  # save weights for next timestep
  model.save_weights('model_weights.h5')
  return model

In [ ]:
def genPaths(monitoring_dates, S0, mu, sigma, N):
  S = []
  for n in range(N):
    path = discretizationScheme(monitoring_dates, S0, mu, sigma)
    S.append(path)
  S = np.matrix(S) # (n,m)
  return S

In [78]:
# generate paths
S = genPaths(monitoring_dates, S0, mu, sigma, N)

# evaluate final time option value
V = np.zeros((N,M))
for n in range(N):
  S_T = S[n,M-1]
  V[n,M-1] = max(K-S_T, 0)

# keep track of Q to see what goes wrong
holdValues = np.zeros((N,M-1))

# no weights to load before the first iteration
load_weights = False

for m in range(M,0,-1):

  # fit the network
  fit_tm = fitBeta(S[:,m-1],V[:,m-1],load_weights=load_weights)

  # load weights next iteration
  load_weights = True

  # preprocess fitted weights for Q
  weights = fit_tm.get_weights()
  w1 = np.array(weights[0][0])
  b1 = np.array(weights[1])
  w2 = np.array(weights[2]).flatten()
  b2 = np.array(weights[3])
  beta_tm = np.concatenate((w1,b1,w2,b2))

  # stop when t=t0
  if m==0:
    break
  for n in range(N):
    Deltat_m = monitoring_dates[m-1] - monitoring_dates[m-2]
    St_mminus1 = S[n,m-2]
    holdValue = Qt_mminus1(beta_tm, St_mminus1, Deltat_m)
    holdValues[n, m-2] = holdValue
    V[n,m-2] = max(St_mminus1 - K, holdValue)


Epoch 1/1000
10/10 [==============================] - 0s 515us/step - loss: 160167.9688
Epoch 2/1000
10/10 [==============================] - 0s 508us/step - loss: 148850.2031
Epoch 3/1000
10/10 [==============================] - 0s 535us/step - loss: 138274.1875
Epoch 4/1000
10/10 [==============================] - 0s 585us/step - loss: 128444.8125
Epoch 5/1000
10/10 [==============================] - 0s 465us/step - loss: 119345.4062
Epoch 6/1000
10/10 [==============================] - 0s 499us/step - loss: 110978.6250
Epoch 7/1000
10/10 [==============================] - 0s 548us/step - loss: 103340.0547
Epoch 8/1000
10/10 [==============================] - 0s 535us/step - loss: 96266.6719
Epoch 9/1000
10/10 [==============================] - 0s 558us/step - loss: 89701.6172
Epoch 10/1000
10/10 [==============================] - 0s 470us/step - loss: 83652.6406
Epoch 11/1000
10/10 [==============================] - 0s 495us/step - loss: 78097.8438
Epoch 12/1000
10/10 [=============

Epoch 1/1000
10/10 [==============================] - 0s 505us/step - loss: 5.7582
Epoch 2/1000
10/10 [==============================] - 0s 460us/step - loss: 5.4243
Epoch 3/1000
10/10 [==============================] - 0s 452us/step - loss: 5.3598
Epoch 4/1000
10/10 [==============================] - 0s 447us/step - loss: 5.3554
Epoch 5/1000
10/10 [==============================] - 0s 445us/step - loss: 5.3425
Epoch 6/1000
10/10 [==============================] - 0s 432us/step - loss: 5.3205
Epoch 7/1000
10/10 [==============================] - 0s 439us/step - loss: 5.3110
Epoch 8/1000
10/10 [==============================] - 0s 431us/step - loss: 5.3019
Epoch 9/1000
10/10 [==============================] - 0s 410us/step - loss: 5.2845
Epoch 10/1000
10/10 [==============================] - 0s 404us/step - loss: 5.2767
Epoch 11/1000
10/10 [==============================] - 0s 415us/step - loss: 5.2578
Epoch 12/1000
10/10 [==============================] - 0s 412us/step - loss: 5.2398
E

Epoch 1/1000
10/10 [==============================] - 0s 491us/step - loss: 0.5691
Epoch 2/1000
10/10 [==============================] - 0s 463us/step - loss: 0.1806
Epoch 3/1000
10/10 [==============================] - 0s 468us/step - loss: 0.0756
Epoch 4/1000
10/10 [==============================] - 0s 451us/step - loss: 0.0226
Epoch 5/1000
10/10 [==============================] - 0s 461us/step - loss: 0.0080
Epoch 6/1000
10/10 [==============================] - 0s 479us/step - loss: 0.0031
Epoch 7/1000
10/10 [==============================] - 0s 458us/step - loss: 0.0012
Epoch 8/1000
10/10 [==============================] - 0s 451us/step - loss: 4.0243e-04
Epoch 9/1000
10/10 [==============================] - 0s 460us/step - loss: 1.3876e-04
Epoch 10/1000
10/10 [==============================] - 0s 436us/step - loss: 5.9400e-05
Epoch 11/1000
10/10 [==============================] - 0s 440us/step - loss: 2.3218e-05
Epoch 12/1000
10/10 [==============================] - 0s 417us/step 

Epoch 1/1000
10/10 [==============================] - 0s 487us/step - loss: 0.5099
Epoch 2/1000
10/10 [==============================] - 0s 475us/step - loss: 0.1509
Epoch 3/1000
10/10 [==============================] - 0s 477us/step - loss: 0.0650
Epoch 4/1000
10/10 [==============================] - 0s 447us/step - loss: 0.0220
Epoch 5/1000
10/10 [==============================] - 0s 454us/step - loss: 0.0073
Epoch 6/1000
10/10 [==============================] - 0s 444us/step - loss: 0.0026
Epoch 7/1000
10/10 [==============================] - 0s 436us/step - loss: 9.4394e-04
Epoch 8/1000
10/10 [==============================] - 0s 439us/step - loss: 3.7588e-04
Epoch 9/1000
10/10 [==============================] - 0s 441us/step - loss: 1.5552e-04
Epoch 10/1000
10/10 [==============================] - 0s 440us/step - loss: 4.5534e-05
Epoch 11/1000
10/10 [==============================] - 0s 768us/step - loss: 1.7734e-05
Epoch 12/1000
10/10 [==============================] - 0s 465us/s

Epoch 1/1000
10/10 [==============================] - 0s 490us/step - loss: 0.5549
Epoch 2/1000
10/10 [==============================] - 0s 462us/step - loss: 0.1750
Epoch 3/1000
10/10 [==============================] - 0s 445us/step - loss: 0.0733
Epoch 4/1000
10/10 [==============================] - 0s 451us/step - loss: 0.0220
Epoch 5/1000
10/10 [==============================] - 0s 527us/step - loss: 0.0077
Epoch 6/1000
10/10 [==============================] - 0s 451us/step - loss: 0.0030
Epoch 7/1000
10/10 [==============================] - 0s 550us/step - loss: 0.0011
Epoch 8/1000
10/10 [==============================] - 0s 447us/step - loss: 3.8269e-04
Epoch 9/1000
10/10 [==============================] - 0s 459us/step - loss: 1.3326e-04
Epoch 10/1000
10/10 [==============================] - 0s 439us/step - loss: 5.8032e-05
Epoch 11/1000
10/10 [==============================] - 0s 446us/step - loss: 2.1204e-05
Epoch 12/1000
10/10 [==============================] - 0s 445us/step 

Epoch 1/1000
10/10 [==============================] - 0s 511us/step - loss: 0.5395
Epoch 2/1000
10/10 [==============================] - 0s 440us/step - loss: 0.1692
Epoch 3/1000
10/10 [==============================] - 0s 478us/step - loss: 0.0711
Epoch 4/1000
10/10 [==============================] - 0s 471us/step - loss: 0.0216
Epoch 5/1000
10/10 [==============================] - 0s 466us/step - loss: 0.0074
Epoch 6/1000
10/10 [==============================] - 0s 452us/step - loss: 0.0028
Epoch 7/1000
10/10 [==============================] - 0s 472us/step - loss: 0.0010
Epoch 8/1000
10/10 [==============================] - 0s 463us/step - loss: 3.6247e-04
Epoch 9/1000
10/10 [==============================] - 0s 475us/step - loss: 1.3434e-04
Epoch 10/1000
10/10 [==============================] - 0s 442us/step - loss: 6.0260e-05
Epoch 11/1000
10/10 [==============================] - 0s 440us/step - loss: 1.9876e-05
Epoch 12/1000
10/10 [==============================] - 0s 444us/step 

Epoch 1/1000
10/10 [==============================] - 0s 497us/step - loss: 0.5204
Epoch 2/1000
10/10 [==============================] - 0s 554us/step - loss: 0.1599
Epoch 3/1000
10/10 [==============================] - 0s 450us/step - loss: 0.0681
Epoch 4/1000
10/10 [==============================] - 0s 446us/step - loss: 0.0213
Epoch 5/1000
10/10 [==============================] - 0s 427us/step - loss: 0.0072
Epoch 6/1000
10/10 [==============================] - 0s 421us/step - loss: 0.0027
Epoch 7/1000
10/10 [==============================] - 0s 438us/step - loss: 9.7335e-04
Epoch 8/1000
10/10 [==============================] - 0s 429us/step - loss: 3.4296e-04
Epoch 9/1000
10/10 [==============================] - 0s 418us/step - loss: 1.3877e-04
Epoch 10/1000
10/10 [==============================] - 0s 436us/step - loss: 5.8029e-05
Epoch 11/1000
10/10 [==============================] - 0s 436us/step - loss: 1.6420e-05
Epoch 12/1000
10/10 [==============================] - 0s 433us/s

Epoch 1/1000
10/10 [==============================] - 0s 504us/step - loss: 0.5335
Epoch 2/1000
10/10 [==============================] - 0s 445us/step - loss: 0.1695
Epoch 3/1000
10/10 [==============================] - 0s 466us/step - loss: 0.0709
Epoch 4/1000
10/10 [==============================] - 0s 450us/step - loss: 0.0212
Epoch 5/1000
10/10 [==============================] - 0s 465us/step - loss: 0.0074
Epoch 6/1000
10/10 [==============================] - 0s 440us/step - loss: 0.0029
Epoch 7/1000
10/10 [==============================] - 0s 464us/step - loss: 0.0011
Epoch 8/1000
10/10 [==============================] - 0s 437us/step - loss: 3.7151e-04
Epoch 9/1000
10/10 [==============================] - 0s 442us/step - loss: 1.2957e-04
Epoch 10/1000
10/10 [==============================] - 0s 439us/step - loss: 5.6473e-05
Epoch 11/1000
10/10 [==============================] - 0s 429us/step - loss: 2.0514e-05
Epoch 12/1000
10/10 [==============================] - 0s 445us/step 

In [ ]:
print(V[4,:])

[ 0.         -0.68446012 -0.64350011 -1.19734301 -1.26412574 -1.51620616
 -0.66629803  0.06071546 -1.83464223  0.        ]


In [ ]:
print(S[4,:])

[[40.         40.68446012 40.64350011 41.19734301 41.26412574 41.51620616
  40.66629803 39.93928454 41.83464223 42.18321741]]


In [ ]:
print(holdValues[4,:])

[-44.31999289 -48.12191997 -47.97233602 -51.01726924 -51.36393994
 -52.81591354 -47.96087768 -43.90256344 -67.70142948]
